In [1]:
#|default_exp autoencoders
LOWER_EPSILON = 1e-20
BIGGER_LOWER_EPSILON = 1e-12
BIGGEST_LOWER_EPSILON = 1e-10
UPPER_EPSILON = 1e20
SMALLER_UPPER_EPSILON = 1e12

In [2]:
#|export
"""
THIS FILE WAS TAKEN FROM https://github.com/BorgwardtLab/topological-autoencoders
"""


"""Base class for autoencoder models."""

import abc
from typing import Dict, Tuple
import torch.nn as nn
class AutoencoderModel(nn.Module, metaclass=abc.ABCMeta):
    """Abstract base class for autoencoders."""

    # pylint: disable=W0221
    @abc.abstractmethod
    def forward(self, x) -> Tuple[float, Dict[str, float]]:
        """Compute loss for model.

        Args:
            x: Tensor with data

        Returns:
            Tuple[loss, dict(loss_component_name -> loss_component)]

        """

    @abc.abstractmethod
    def encode(self, x):
        """Compute latent representation."""

    @abc.abstractmethod
    def decode(self, z):
        """Compute reconstruction."""

    def immersion(self, x):
        """The immersion defined by the autoencoder (or more precisely, the decoder)."""
        return self.decoder(x)


In [3]:
#|export
"""
THIS FILE WAS PARTLY TAKEN FROM https://github.com/BorgwardtLab/topological-autoencoders
"""


"""Submodules used by models."""


# Hush the linter: Warning W0221 corresponds to a mismatch between parent class
# method signature and the child class
# pylint: disable=W0221

import numpy as np
import torch
import torch.nn as nn
from torch.distributions import Normal
import torch.nn.functional as F
from collections import OrderedDict

class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)


class Print(nn.Module):
    def __init__(self, name):
        self.name = name
        super().__init__()

    def forward(self, x):
        print(self.name, x.size())
        return x


class BoxAutoEncoder(AutoencoderModel):
    """100-100-100-2-100-100-100."""

    def __init__(self, input_dims=(1, 28, 28), **kwargs):
        super().__init__()
        self.latent_dim = 2
        n_input_dims = np.prod(input_dims)
        self.input_dim = n_input_dims.item()
        self.input_dims = input_dims

        self.encoder = nn.Sequential(
            # View((-1, n_input_dims)),
            nn.ELU(),
            nn.Linear(self.input_dim, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=self.latent_dim),
        )

        self.decoder = nn.Sequential(
            nn.ELU(),
            nn.Linear(self.latent_dim, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=self.input_dim),
            # View((-1,) + tuple(input_dims)),
        )

        self.reconst_error = nn.MSELoss()

        self.register_hook()

    def encode(self, x):
        """Compute latent representation using convolutional autoencoder."""
        x = x.view((-1, self.input_dim))

        x = self.encoder(x)
        return x

    def decode(self, z):
        """Compute reconstruction using convolutional autoencoder."""
        z = self.decoder(z)
        z = z.view((-1, *self.input_dims))
        return z

    def forward_(self, x):
        x = x.view((-1, self.input_dim))
        x = self.encoder(x)

        z = self.decoder(x)
        z = z.view((-1, *self.input_dims))

        return z

    def forward(self, x):
        """Apply autoencoder to batch of input images.

        Args:
            x: Batch of images with shape [bs x channels x n_row x n_col]

        Returns:
            tuple(reconstruction_error, dict(other errors))

        """
        latent = self.encode(x)
        x_reconst = self.decode(latent)
        reconst_error = self.reconst_error(x, x_reconst)
        return reconst_error, {'reconstruction_error': reconst_error}

    def register_hook(self):
        self.encoder.register_forward_hook(self.get_activation())

    def get_activation(self):
        """
        :return: activations at layer model
        """

        def hook(model, input, output):
            self.latent_activations = output
            self.latent_activations_detached = output.detach()

        return hook

    def load(self, path):
        dict = torch.load(path)

        # edit keys, since they are created from higher TopoAE class
        new_dict = OrderedDict([])

        for key in dict.keys():
            arr = key.split(".")
            if len(arr) == 1 or arr[-1] not in ["weight", "bias"]:
                continue
                # del dict[key]
            # elif arr[-1] not in ["weight", "bias"]:
            #    del dict[key]
            else:
                if arr[0] == "autoencoder":
                    new_key = ".".join(arr[1:])
                else:
                    new_key = key

                new_dict.update({new_key: dict[key]})

                # dict = OrderedDict([(new_key, v) if k == key else (k, v) for k, v in d.items()])
                # new_dict.update({new_key: dict[key]})

        # self.load_state_dict(torch.load(path))
        self.load_state_dict(new_dict)

        self.eval()


class ConvolutionalAutoEncoder(AutoencoderModel):
    """Convolutional autoencoder for MNIST and FashionMNIST"""

    def __init__(self, input_dims=(1, 28, 28), **kwargs):
        super().__init__()
        self.latent_dim = 2
        n_input_dims = np.prod(input_dims)
        self.input_dim = n_input_dims.item()
        self.input_dims = input_dims

        self.encoder = nn.Sequential(
            # b, 1, 28, 28
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ELU(),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ELU(),
            nn.MaxPool2d(2, stride=1),  # b, 8, 2, 2
            nn.Conv2d(8, 2, 2, stride=1, padding=0),  # b, 2, 1, 1
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(2, 8, 2, stride=1),  # b, 8, 2, 2
            nn.ELU(),
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ELU(),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ELU(),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

        self.reconst_error = nn.MSELoss()

        self.register_hook()

    def immersion(self, x):
        return self.decoder(x).view(-1, 784)

    def encode(self, x):
        """Compute latent representation using convolutional autoencoder."""
        return self.encoder(x)

    def decode(self, z):
        """Compute reconstruction using convolutional autoencoder."""
        return self.decoder(z)

    def forward_(self, x):
        x = self.encoder(x)
        z = self.decoder(x)

        return z

    def forward(self, x):
        """Apply autoencoder to batch of input images.

        Args:
            x: Batch of images with shape [bs x channels x n_row x n_col]

        Returns:
            tuple(reconstruction_error, dict(other errors))

        """
        latent = self.encode(x)
        x_reconst = self.decode(latent)
        reconst_error = self.reconst_error(x, x_reconst)
        return reconst_error, {'reconstruction_error': reconst_error}

    def register_hook(self):
        self.encoder.register_forward_hook(self.get_activation())

    def get_activation(self):
        """
        :return: activations at layer model
        """

        def hook(model, input, output):
            self.latent_activations = output
            self.latent_activations_detached = output.detach()

        return hook

    def load(self, path):
        dict = torch.load(path)

        # edit keys, since they are created from higher TopoAE class
        new_dict = OrderedDict([])

        for key in dict.keys():
            arr = key.split(".")
            if len(arr) == 1 or arr[-1] not in ["weight", "bias"]:
                continue
                # del dict[key]
            # elif arr[-1] not in ["weight", "bias"]:
            #    del dict[key]
            else:
                if arr[0] == "autoencoder":
                    new_key = ".".join(arr[1:])
                else:
                    new_key = key

                new_dict.update({new_key: dict[key]})

                # dict = OrderedDict([(new_key, v) if k == key else (k, v) for k, v in d.items()])
                # new_dict.update({new_key: dict[key]})

        # self.load_state_dict(torch.load(path))
        self.load_state_dict(new_dict)

        self.eval()


class LinearAE(AutoencoderModel):
    """input dim - 2 - input dim."""

    def __init__(self, input_dims=(1, 28, 28)):
        super().__init__()
        self.input_dims = input_dims
        n_input_dims = np.prod(input_dims)
        self.encoder = nn.Sequential(
            View((-1, n_input_dims)),
            nn.Linear(n_input_dims, 2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, n_input_dims),
            View((-1,) + tuple(input_dims)),
        )
        self.reconst_error = nn.MSELoss()

    def encode(self, x):
        """Compute latent representation using convolutional autoencoder."""
        return self.encoder(x)

    def decode(self, z):
        """Compute reconstruction using convolutional autoencoder."""
        return self.decoder(z)

    def forward(self, x):
        """Apply autoencoder to batch of input images.

        Args:
            x: Batch of images with shape [bs x channels x n_row x n_col]

        Returns:
            tuple(reconstruction_error, dict(other errors))

        """
        latent = self.encode(x)
        x_reconst = self.decode(latent)
        reconst_error = self.reconst_error(x, x_reconst)
        return reconst_error, {'reconstruction_error': reconst_error}


In [4]:
"""
The Determinant Regularizer
"""

import torch

from autometric.manifolds import RiemannianManifold
from autometric.metrics import PullbackMetric
from autometric.connections import LeviCivitaConnection
LOWER_EPSILON = 1e-12
UPPER_EPSILON = 1e-3


class Loss:
    """
    A Basis class for custom loss functions
    """
    def __init__(self, model=None):
        # a manifold object
        self.model = model
        pbm = PullbackMetric(2, model.immersion)
        lcc = LeviCivitaConnection(2, pbm)
        self.manifold = RiemannianManifold(2, (1, 1), metric=pbm, connection=lcc)

    @staticmethod
    def sample_points(latent_activations=None, outputs=None, num_samples=1):
        """
        Randomly sample points from batch in latent space and map it to the output space
        :param outputs: batch in output space
        :param latent_activations: batch in latent space
        :param num_samples: number of samples to take
        :return: (points in latent space, points in output space)
        """

        # randomly sample the points
        rand_choice = torch.randperm(latent_activations.shape[0])[:num_samples]

        pimg_origin = latent_activations[rand_choice, :]

        if outputs is None:
            return pimg_origin

        img_origin = outputs[rand_choice, :]

        return img_origin, pimg_origin


class DeterminantLoss(Loss):
    def __call__(self, epoch=0, *args, **kwargs):
        """
            Our Determinant Regularizer
        Args:
            epoch: current epoch
        Returns:

        """

        # here you can control whether the regularizer should only be switched on after a certain epoch
        if epoch >= 0:
            loss_det = self.determinant_loss()
        else:
            loss_det = torch.tensor([0.], device=device, requires_grad=True)

        return loss_det

    def determinant_loss(self):
        """
        Calculate the actual loss
        Returns:
            The determinant loss
        """

        # calculate the logarithm of the generalized jacobian determinant
        log_dets = self.manifold.metric_logdet(base_point=self.model.latent_activations)

        # replace nan values with a small number
        #EPSILON = 1e-9
        #torch.nan_to_num(log_dets, nan=EPSILON, posinf=EPSILON, neginf=EPSILON)
        torch.nan_to_num(log_dets, nan=1., posinf=1., neginf=1.)

        # calculate the variance of the logarithm of the generalized jacobian determinant
        raw_loss = torch.var(log_dets)

        return raw_loss


INFO: Using numpy backend


In [6]:
#|export
"""
THIS FILE WAS PARTLY TAKEN FROM https://github.com/BorgwardtLab/topological-autoencoders
"""
"""Vanilla models."""

class VanillaAutoencoderModel(AutoencoderModel):
    def __init__(self, autoencoder_model='ConvolutionalAutoencoder',
                 ae_kwargs=None):
        super().__init__()

        ae_kwargs = ae_kwargs if ae_kwargs else {}
        self.autoencoder = getattr(submodules, autoencoder_model)(**ae_kwargs)

        self.with_geom_loss = False

        if self.with_geom_loss:
            self.determinant_criterion = DeterminantLoss(model=self.autoencoder)

    def forward(self, x):
        # Use reconstruction loss of autoencoder

        if self.with_geom_loss:
            ae_loss, ae_loss_comp = self.autoencoder(x)

            det_loss = self.determinant_criterion().detach_()

            loss = ae_loss
            loss_components = {
                'loss.autoencoder': ae_loss,
                'loss.geom_error': det_loss
            }

            loss_components.update(ae_loss_comp)
            return (
                loss,
                loss_components
            )
        else:
            return self.autoencoder(x)

    def encode(self, x):
        return self.autoencoder.encode(x)

    def decode(self, z):
        return self.autoencoder.decode(z)


# 2a Autoencoders
A significant portion of the training machinery from the original repo can be subsumed by the unwitting use of a *framework!* Yes, frameworks - factory factory factories and all, we love them.
The only nonstandard bit from the bespoke class above which affects the downstream geometry is using `immersion` as an alias for `decoder`.

In [ ]:
#|export
import torch
import torch.nn as nn
import pytorch_lightning as pl

class DerrickTheAutoencoder(pl.LightningModule):
    """
    Derrick doesn't have many friends, so he spends his time encoding data into the specified latent dimension using tanh activations.
    Fortunately, he's friendly with PyTorch Lightning; he gets struck occasionally, but that's made up by his lickety-split training.
    """
    def __init__(self, input_dim, intrinsic_dimension, learning_rate=1e-3):
        super().__init__()
        self.intrinsic_dimension = intrinsic_dimension
        self.lr = learning_rate
        self.encoder = nn.Sequential(
            nn.Linear(in_features=input_dim, out_features=256),
            nn.ELU(),
            nn.Linear(in_features=256, out_features=128),
            nn.ELU(),
            nn.Linear(in_features=128, out_features=64),
            nn.ELU(),
            nn.Linear(in_features=64, out_features=intrinsic_dimension)
        )
        self.decoder = nn.Sequential(
            nn.Linear(in_features=intrinsic_dimension, out_features=64),
            nn.ELU(),
            nn.Linear(in_features=64, out_features=128),
            nn.ELU(),
            nn.Linear(in_features=128, out_features=256),
            nn.ELU(),
            nn.Linear(in_features=256, out_features=input_dim)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def immersion(self, x):
        """The immersion defined by the autoencoder (or more precisely, the decoder)."""
        return self.decoder(x)

    def training_step(self, batch, batch_idx):
        x = batch
        x_hat = self(x)
        loss = nn.MSELoss()(x_hat, x)
        self.log('train_loss', loss, prog_bar=True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x = batch
        x_hat = self(x)
        loss = nn.MSELoss()(x_hat, x)
        self.log('val_loss', loss, prog_bar=True, on_epoch=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x = batch
        x_hat = self(x)
        loss = nn.MSELoss()(x_hat, x)
        self.log('test_loss', loss)
        return loss
    
    

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer
